In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
import statsmodels.formula.api as sm
#import matplotlib.pyplot as plt
import statsmodels.stats.outliers_influence as sif
import time
#import matplotlib.pyplot as plt
from dataiku import pandasutils as pdu

# Read recipe inputs
overall_final_modelling_data = dataiku.Dataset("overall_final_modelling_data")
overall_final_modelling_data_df = overall_final_modelling_data.get_dataframe()


#################################################################################################################
# Declare type by which user want to select the curvature and Lambda value (assign Loop_Type as 'Automated' to get curvature and lambda from code)

Loop_Type = 'Automated'

# declare the number of sales lag and addstock to be calculated below
var_sales_lags = 3
var_promo_lags = 2

# declare the starting month of the modelling time
var_PERIOD_START = 201907

# declare all variables
# for lags and addstock call
var_CHANNEL =   ['total_calls','total_200_syr_sample','total_300_syr_sample','total_speaker']

#additional variable for COVID or market event or seasonality variable
special_var = 'mobility_index'

# for Impact calculation adding the COVID or market event or seasonality variable
var_CHANNEL_1 =   ['total_calls','total_200_syr_sample','total_300_syr_sample','total_speaker','mobility_index']

#Enter the cost of each promotion channel and NRx cost
var_prom_cost_unit_final = [250,76,60,100] #enter the promotion unit price for each promotion channel in the same order as var_CHANNEL is declared
var_sup = 1097  #per unit price of Rx
#######################################################################################################################

# combining sales and sample variable

overall_final_modelling_data_df['total_sales']= overall_final_modelling_data_df['overall_nrx']
overall_final_modelling_data_df['total_200_syr_sample']= overall_final_modelling_data_df['overall_samples_200_syr']
overall_final_modelling_data_df['total_300_syr_sample']= overall_final_modelling_data_df['overall_samples_300_syr']
overall_final_modelling_data_df['total_calls']= overall_final_modelling_data_df['overall_calls']
overall_final_modelling_data_df['total_speaker']= overall_final_modelling_data_df['overall_speaker']

overall_final_modelling_data_df.fillna(0, inplace=True)

In [0]:
#main code for MMX


#Sorting data on HCP ID and YearMonth
overall_final_modelling_data_df.sort_values(['regid', 'month_id'], inplace=True)

#Function to calculate sales and promotional lags
def lag(overall_final_modelling_data_df, var, group_var, new_var,j) :
    overall_final_modelling_data_df[new_var] = overall_final_modelling_data_df.groupby(group_var)[var].shift(j)
    overall_final_modelling_data_df[new_var].fillna(0, inplace=True)
    return overall_final_modelling_data_df

#Calculating target variable (i.e., sales) lags on input data (add the lag range in the above part of the code)
for i in range(1, var_sales_lags+1) :
    new_variable = 'total_sales' + "_" + str(i)
    overall_final_modelling_data_df = lag(overall_final_modelling_data_df, 'total_sales', 'regid', new_variable, i)


#Calculating promotional lags for various channels on input data
for i in range(0,len(var_CHANNEL)) :
    for j in range(1,var_promo_lags+1) :
        new_variable=var_CHANNEL[i]+"_"+str(j)
        overall_final_modelling_data_df=lag(overall_final_modelling_data_df, var_CHANNEL[i], 'regid', new_variable, j)

#Filtering input data on the basis of analysis period
model_data = overall_final_modelling_data_df[overall_final_modelling_data_df['month_id'] >= var_PERIOD_START]

#Getting list of unique segments
segments = list(model_data['segment'].unique())

#Getting list of unique year-months
months=list(model_data['month_id'].unique())

In [0]:
#Getting step-size for increasing curvature values


#Storing curvature values for different channel segment combination in a dataf

C_channel_seg={}
C_channel_seg['Segment']=[]
for i in range(0,len(segments)) :
    for j in range(0,4) :
        C_channel_seg['Segment'].append(segments[i])

for i in range(0,len(var_CHANNEL)) :
    new_var="C_"+var_CHANNEL[i]
    C_channel_seg[new_var]=[]
    for j in range(0,len(segments)) :
        channel_segment_mean=model_data.loc[(model_data['segment']==segments[j]) & (model_data[var_CHANNEL[i]]>0),var_CHANNEL[i]].mean()
        mean_range=[x/channel_segment_mean for x in range(1,5)]
        C_channel_seg[new_var].extend(mean_range)

Curvature_channel_seg=pd.DataFrame(C_channel_seg)

In [0]:
print(C_channel_seg)

In [0]:
############# Calculating Best C Value for Channel Segment Combination ##################
import statsmodels.api as sm

Coeff_Final=pd.DataFrame()

#selecting a segment
for i in range(0,len(segments)) :
    #subsetting model data for a segment
    seg_data=model_data[model_data['segment']==segments[i]]

    for j in range(0,len(var_CHANNEL)) :
        for l in range(1,5) :
            adstock=seg_data[var_CHANNEL[j]]
            for lam in np.arange(.1,.9,.1):
                for k in range(1,var_promo_lags+1) :

                    channel_lag=var_CHANNEL[j]+"_"+str(k)
                    adstock = adstock + (lam**k)*seg_data[channel_lag] #.33 is the the lambda value, we can take it as user input in future

                var='C_'+var_CHANNEL[j]
                Curvature=[]
                Curv=Curvature_channel_seg.loc[Curvature_channel_seg['Segment']==segments[i],var]
                Curv=pd.DataFrame(Curv)
                Curv=Curv.iloc[l-1,:]
                C=pd.DataFrame(Curv)
                Curvature.append(Curv)
                seg_data[var_CHANNEL[j]+'_t']=1-np.exp(-1*Curv[0]*adstock)
                X_Sales_Lag_Cols=['total_sales' + '_' +
                                str(lag_num) for lag_num in
                                range(1, var_sales_lags+1)]
                X_Sales_Lag_Cols.append(var_CHANNEL[j] + '_t')
                Y_Cols = ['total_sales']
                X = seg_data[X_Sales_Lag_Cols]
                Y = seg_data[Y_Cols]
                X = sm.add_constant(X)
                model = sm.OLS(Y, X).fit()
                fit_summary = pd.DataFrame(model.summary2().tables[1])
                fit_summary = fit_summary.reset_index()
                fit_summary_2 = pd.DataFrame(model.summary2().tables[0])
                Coeff = fit_summary[['index', 'Coef.', 'Std.Err.']]
                Coeff['adj r2'] = fit_summary_2.iloc[0,3]
                Coeff['Segment'] = segments[i]
                Coeff['Curavture'] = C.iloc[0,0]
                Coeff['Lambda'] = lam
                Coeff.columns = ['Variable', 'Coeff', 'P-Value', 'Adj R-Sq',
                                   'Segment', 'Curvature','Lambda']
                #Appending model_summaries
                Coeff_Final = pd.concat([Coeff_Final, Coeff], axis=0)
#print(Coeff_Final)

In [0]:
print(Coeff_Final)

In [0]:
###################### Selecting Best C Value for channel segment combination ##############################

channel_name_transformed=[]
for i in var_CHANNEL :
    tranformed_channel_name=i+'_t'
    channel_name_transformed.append(tranformed_channel_name)
Coeff_Final=Coeff_Final[Coeff_Final['Variable'].isin(channel_name_transformed)]
Coeff_Final.reset_index(drop=True,inplace=True)

Coeff_Best_Model = Coeff_Final[Coeff_Final['Adj R-Sq'] == Coeff_Final.groupby(['Segment','Variable'])['Adj R-Sq'].transform(max)]
Coeff_Best_Model = Coeff_Final[Coeff_Final['P-Value'] == Coeff_Final.groupby(['Segment','Variable'])['P-Value'].transform(min)]
Coeff_Best_Model = Coeff_Best_Model[Coeff_Best_Model['Curvature'] == Coeff_Best_Model.groupby(['Segment','Variable'])['Curvature'].transform(min)]
Coeff_Best_Model = Coeff_Best_Model[Coeff_Best_Model['Lambda'] == Coeff_Best_Model.groupby(['Segment','Variable'])['Lambda'].transform(max)]
Coeff_Best_Model.reset_index(drop=True,inplace=True)
Coeff_Best_Model=Coeff_Best_Model[['Variable','Segment','Adj R-Sq','Curvature','Lambda']]
#Channel_Equation=pd.DataFrame(zip(var_CHANNEL,equation_final),
#                              columns=['Variable','Equation'])

#Creating flag in overall curvature values
#Coeff_Final_1 = pd.merge(Coeff_Best_Model,Coeff_Final, how='left',on=['Variable','Segment','Curvature','Lambda'])
#Coeff_Final_1['Flag']=1
# Coeff_Final = pd.merge(Coeff_Final_1,Coeff_Final, how='outer',on=['Variable','Segment','Curvature','Lambda'])
# del Coeff_Final['Coeff_x'],Coeff_Final['P-Value_x'],Coeff_Final['Adj R-Sq_x']
# Coeff_Final.columns = ['Variable', 'Segment', 'Curvature', 'Lambda','Flag', 'Coeff', 'P-Value', 'Adj R-Sq']
# Coeff_Final.fillna(0, inplace=True)

In [0]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [0]:
#######################  Running Models #####################################################

################### Calcualating Parameter Estimates ########################################



#Function to generate the linear equation for Curve_fit

if Loop_Type == "Automated" :

    Parameter_Estimates=pd.DataFrame()
    Final_Model_Data=pd.DataFrame()
    vif=pd.DataFrame()

    for i in range(0,len(segments)) :
        seg_data=model_data[model_data['segment']==segments[i]]
        Channel_transformed_names=[]
        for j in range(0,len(var_CHANNEL)) :

            adstock=seg_data[var_CHANNEL[j]]
            variable=var_CHANNEL[j]+'_t'
            lam=Coeff_Best_Model.loc[(Coeff_Best_Model['Segment']==segments[i]) & (Coeff_Best_Model['Variable']==variable),'Lambda']
            lam=pd.DataFrame(lam)
            for k in range(1,var_promo_lags+1) :
                channel_lag=var_CHANNEL[j]+'_'+str(k)
                adstock=adstock + (lam.iloc[0,0]**k)*seg_data[channel_lag]
            Channel_transformed_names.append(variable)
            Curv=Coeff_Best_Model.loc[(Coeff_Best_Model['Segment']==segments[i]) & (Coeff_Best_Model['Variable']==variable),'Curvature']
            Curv=pd.DataFrame(Curv)
            seg_data[variable]=adstock
            seg_data[variable]=1-np.exp(-1*Curv.iloc[0,0]*adstock)


        X_Sales_Lag_Cols=['total_sales' + '_' +
                                str(lag_num) for lag_num in
                                range(1, var_sales_lags+1)]
        X_Sales_Lag_Cols.extend(Channel_transformed_names)
        X_Sales_Lag_Cols.append(special_var)
        Y_Cols=['total_sales']
        X=seg_data[X_Sales_Lag_Cols]
        Y=seg_data[Y_Cols]
        vif['Variable'] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, m) for m in range(X.shape[1])]
        X=sm.add_constant(X)

        model=sm.OLS(Y,X).fit()
        fit_summary = pd.DataFrame(model.summary2().tables[1])
        fit_summary = fit_summary.reset_index()
        fit_summary_2 = pd.DataFrame(model.summary2().tables[0])
        Coeff = fit_summary[['index', 'Coef.', 'Std.Err.']]
        Coeff['adj r2'] = fit_summary_2.iloc[0,3]
        Coeff['R2'] = fit_summary_2.iloc[6,1]
        #Coeff['R2'] = Coeff['R2'].astype(float)
        #Coeff['VIF'] = 1/(1-pow(Coeff['R2'],2))
        Coeff['Segment'] = segments[i]
        Coeff.columns = ['Variable', 'Coeff', 'P-Value', 'Adj R-Sq','R2',
                                   'Segment']

        Coeff_list=list(Coeff['Coeff'])
        X_Variable=list(Coeff['Variable'])
        Coeff = pd.merge(Coeff,vif, how= 'left' ,on = 'Variable')

        Coeff['Coeff']=Coeff_list
        Final_Model_Data=pd.concat([Final_Model_Data,seg_data])  #creating addstock based on best C and lambda
        Parameter_Estimates=pd.concat([Parameter_Estimates,Coeff],axis=0)


    Parameter_Estimates.reset_index(drop=True,inplace=True)
    Parameter_Estimates=Parameter_Estimates[['Variable','Segment','Coeff','P-Value', 'Adj R-Sq','R2','VIF']]

else :
    Coeff_Best_Model_1 = dataiku.Dataset("user_input")
    Coeff_Best_Model_1 = Coeff_Best_Model_1.get_dataframe()
    Parameter_Estimates=pd.DataFrame()
    Final_Model_Data=pd.DataFrame()
    vif=pd.DataFrame()

    for i in range(0,len(segments)) :
        seg_data=model_data[model_data['segment']==segments[i]]
        Channel_transformed_names=[]
        for j in range(0,len(var_CHANNEL)) :

            adstock=seg_data[var_CHANNEL[j]]
            variable=var_CHANNEL[j]+'_t'
            lam=Coeff_Best_Model_1.loc[(Coeff_Best_Model_1['Segment']==segments[i]) & (Coeff_Best_Model_1['Variable']==variable),'Lambda']
            lam=pd.DataFrame(lam)
            for k in range(1,var_promo_lags+1) :
                channel_lag=var_CHANNEL[j]+'_'+str(k)
                adstock=adstock + (lam.iloc[0,0]**k)*seg_data[channel_lag]
            Channel_transformed_names.append(variable)
            Curv=Coeff_Best_Model_1.loc[(Coeff_Best_Model_1['Segment']==segments[i]) & (Coeff_Best_Model_1['Variable']==variable),'Curvature']
            Curv=pd.DataFrame(Curv)
            seg_data[variable]=adstock
            seg_data[variable]=1-np.exp(-1*Curv.iloc[0,0]*adstock)


        X_Sales_Lag_Cols=['total_sales' + '_' +
                                str(lag_num) for lag_num in
                                range(1, var_sales_lags+1)]
        X_Sales_Lag_Cols.extend(Channel_transformed_names)
        X_Sales_Lag_Cols.append(special_var)
        Y_Cols=['total_sales']
        X=seg_data[X_Sales_Lag_Cols]
        Y=seg_data[Y_Cols]
        vif['Variable'] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, m) for m in range(X.shape[1])]
        X=sm.add_constant(X)

        model=sm.OLS(Y,X).fit()
        fit_summary = pd.DataFrame(model.summary2().tables[1])
        fit_summary = fit_summary.reset_index()
        fit_summary_2 = pd.DataFrame(model.summary2().tables[0])
        Coeff = fit_summary[['index', 'Coef.', 'Std.Err.']]
        Coeff['adj r2'] = fit_summary_2.iloc[0,3]
        Coeff['R2'] = fit_summary_2.iloc[6,1]
        #Coeff['R2'] = Coeff['R2'].astype(float)
        #Coeff['VIF'] = 1/(1-pow(Coeff['R2'],2))
        Coeff['Segment'] = segments[i]
        Coeff.columns = ['Variable', 'Coeff', 'P-Value', 'Adj R-Sq','R2',
                                   'Segment']

        Coeff_list=list(Coeff['Coeff'])
        X_Variable=list(Coeff['Variable'])
        Coeff = pd.merge(Coeff,vif, how= 'left' ,on = 'Variable')

        Coeff['Coeff']=Coeff_list
        Final_Model_Data=pd.concat([Final_Model_Data,seg_data])  #creating addstock based on best C and lambda
        Parameter_Estimates=pd.concat([Parameter_Estimates,Coeff],axis=0)


    Parameter_Estimates.reset_index(drop=True,inplace=True)
    Parameter_Estimates=Parameter_Estimates[['Variable','Segment','Coeff','P-Value', 'Adj R-Sq','R2','VIF']]

In [0]:
print(Parameter_Estimates)

In [0]:
######################### Calculating Impact ####################################

######################### 1. Brand Equity #######################################

BE_Impact=pd.DataFrame()
for i in range(0,len(segments)) :
        seg_data=Final_Model_Data[Final_Model_Data['segment']==segments[i]]
        hcp_count=len(seg_data['regid'].unique())
        coeff=Parameter_Estimates.loc[(Parameter_Estimates['Variable']=='const') & (Parameter_Estimates['Segment']==segments[i]),'Coeff']
        sales_lag_coeffs=[]
        for j in range(1, var_sales_lags+1) :
            s = 'total_sales' + '_' + str(j)
            sales_lag_coeffs.extend(list(Parameter_Estimates.loc[((Parameter_Estimates['Segment']==segments[i]) & (Parameter_Estimates['Variable']==s)),'Coeff']))
        Monthly_BEs=[]
        for month in range(0,len(months)) :
            if month == 0 :
                Monthly_BEs.append(coeff.iloc[0])
            else :
                iters = min(month, 3)
                brand_equity = coeff.iloc[0]
                for k in range(0, iters) :
                    brand_equity += (Monthly_BEs[month-1-k] * sales_lag_coeffs[k])
                brand_equity = [brand_equity]
                Monthly_BEs.extend(brand_equity)

        seg_be=pd.DataFrame(zip(months,Monthly_BEs),
                            columns=['month_id', 'Impact'])
        seg_be['Impact'] = seg_be['Impact'] * hcp_count
        seg_be['Segment'] = segments[i]
        seg_be['Channel'] = 'Brand Equity'
        BE_Impact = pd.concat([BE_Impact, seg_be], axis=0)

NL_Months=[]
NL_BE_Impact=[]

for i in months :
        NL_Months.append(i)
        NL_BE_Impact.append(BE_Impact[BE_Impact['month_id']==i]['Impact'].sum())
BE_Impact_NL=pd.DataFrame(zip(NL_Months,NL_BE_Impact),
                              columns=['month_id', 'Impact'])
BE_Impact_NL['Segment']='National'
BE_Impact_NL['Channel'] = 'Brand Equity'
BE_Impact=pd.concat([BE_Impact,BE_Impact_NL],axis=0).reset_index(drop=True)

In [0]:
############################ 2. Carryover #######################################

CO_Impact = pd.DataFrame()
for i in range(0, len(segments)) :

    seg_data=Final_Model_Data[Final_Model_Data['segment'] == segments[i]]


    Previous_Sales=[]
    for j in range(var_sales_lags, 0, -1) :

        s = 'total_sales'+'_'+str(j)
        Previous_Sales.append(seg_data.loc[seg_data['month_id']==var_PERIOD_START, s].sum())

    sales_lag_coeffs=[]
    for m in range(1, var_sales_lags+1) :
        s = 'total_sales'+'_'+str(m)
        sales_lag_coeffs.extend(Parameter_Estimates.loc[((Parameter_Estimates['Segment']==segments[i]) & (Parameter_Estimates['Variable']==s)),'Coeff'])

    for k in range(0,len(months)) :
        c = 0
        for l in range(0, var_sales_lags) :
            c += Previous_Sales[var_sales_lags+k-1-l] * sales_lag_coeffs[l]
        Previous_Sales.append(c)

    Previous_Sales=Previous_Sales[var_sales_lags:]
    seg_impact = pd.DataFrame(zip(months, Previous_Sales),
                              columns =['month_id', 'Impact'])
    seg_impact['Segment'] = segments[i]
    seg_impact['Channel'] = 'CarryOver'
    CO_Impact = pd.concat([CO_Impact, seg_impact], axis=0)


NL_Months=[]
NL_CO_Impact=[]
for i in months :
    NL_Months.append(i)
    NL_CO_Impact.append(CO_Impact[CO_Impact['month_id']==i]['Impact'].sum())

CO_Impact_NL=pd.DataFrame(zip(NL_Months,NL_CO_Impact),
                          columns=['month_id', 'Impact'])
CO_Impact_NL['Segment']='National'
CO_Impact_NL['Channel'] = 'CarryOver'
CO_Impact=pd.concat([CO_Impact,CO_Impact_NL],axis=0).reset_index(drop=True)

In [0]:
Final_Model_Data.head(n=3)

In [0]:
############################# 3. CHANNEL IMPACT #############################################

Prom_Impact=pd.DataFrame()

for i in range(0,len(segments)) :
    seg_data = Final_Model_Data[Final_Model_Data['segment'] == segments[i]]
    for j in range(0,len(var_CHANNEL_1)) :
        if var_CHANNEL_1[j]==special_var :
            s=var_CHANNEL_1[j]
        else:
            s=var_CHANNEL_1[j]+"_t"

        coeff = Parameter_Estimates.loc[(Parameter_Estimates['Segment'] == segments[i]) & (Parameter_Estimates['Variable'] == s) , 'Coeff']
        sales_lag_coeffs = []
        for m in range(1, var_sales_lags + 1) :
            lag_var = 'total_sales' + '_' + str(m)
            sales_lag_coeffs.extend(list(Parameter_Estimates.loc[((Parameter_Estimates['Segment'] == segments[i]) & (Parameter_Estimates['Variable'] == lag_var)), 'Coeff']))

        Monthly_Impacts=[]
        for month in range(0, len(months)) :
            total_200_syr_sample=seg_data[seg_data['month_id'] == months[month]][s].sum()
            if month == 0 :

                impact = coeff * total_200_syr_sample
                Monthly_Impacts.extend(impact)
            else :

                iters = min(month, var_sales_lags)
                impact = coeff.iloc[0] * total_200_syr_sample
                for z in range(0, iters) :

                    impact += (Monthly_Impacts[month-1-z] * sales_lag_coeffs[z])
                impact = [impact]
                Monthly_Impacts.extend(impact)

        seg_impact = pd.DataFrame(zip(months, Monthly_Impacts),
                          columns =['month_id', 'Impact'])
        seg_impact['Segment'] = segments[i]
        seg_impact['Channel'] = var_CHANNEL_1[j]
        Prom_Impact = pd.concat([Prom_Impact, seg_impact], axis=0)


Prom_Impact_NL=Prom_Impact.groupby(['month_id','Channel'])['Impact'].sum().reset_index()
Prom_Impact_NL.sort_values(['Channel'],inplace=True)
Prom_Impact_NL['Segment']='National'
Prom_Impact_NL=Prom_Impact_NL[['month_id','Impact','Segment','Channel']]
Prom_Impact=pd.concat([Prom_Impact,Prom_Impact_NL],axis=0).reset_index(drop=True)

In [0]:
    # Combining BE,CO and Promotional Impact into a single dataframe
Final_Impact=(BE_Impact.append(CO_Impact)).append(Prom_Impact)
Final_Impact.reset_index(drop=True,inplace=True)

Overall_Impact = Final_Impact[Final_Impact['Segment']=='National']

In [0]:
################################### Data With Impacts #####################################

Final_Model_Data['const']=1
Coeff_names=list(Parameter_Estimates['Variable'].unique())
Model_Data_With_Impact=pd.DataFrame()
for i in range(0,len(segments)) :
    seg_data=Final_Model_Data[Final_Model_Data['segment']==segments[i]]
    for j in range(0,len(Coeff_names)) :
        var=Coeff_names[j]+"_Impact"
        Coeff=Parameter_Estimates.loc[(Parameter_Estimates['Segment']==segments[i]) & (Parameter_Estimates['Variable']==Coeff_names[j]),'Coeff']
        seg_data[var]=seg_data[Coeff_names[j]]*Coeff.iloc[0]
    Model_Data_With_Impact=pd.concat([Model_Data_With_Impact,seg_data],axis=0)

Model_Data_With_Impact=Model_Data_With_Impact.loc[:,Model_Data_With_Impact.columns.str.contains('_Impact') | Model_Data_With_Impact.columns.isin(['regid','month_id'])]

Model_Data_With_Impact_Final= Model_Data_With_Impact
Model_Data_With_Impact_Final.fillna(0, inplace=True)
carryover=0
for i in range(1,var_sales_lags+1) :
    var='total_sales'+'_'+str(i)+'_Impact'
    carryover+=Model_Data_With_Impact_Final[var]
Model_Data_With_Impact_Final['Carryover']=carryover

In [0]:
print(Final_Impact)

In [0]:
####################### Summary ################################

Seg_Channel_Hist_Dist=pd.DataFrame()
for i in range(0,len(segments)) :
    for j in range(0,len(var_CHANNEL_1)) :
        Segment=segments[i]
        Channel=var_CHANNEL_1[j]
        seg_data=Final_Model_Data[Final_Model_Data['segment']==segments[i]]
        hcp_count=seg_data['regid'].nunique()
        Months=len(months)
        Promotion=seg_data[var_CHANNEL_1[j]].sum()
        Sales=seg_data['total_sales'].sum()
        Hist_Dist=pd.DataFrame({'Segment':[Segment],'Channel':[Channel],'HCP Count':[hcp_count],
                                   'Channel_Promotion':[Promotion],'Sales':[Sales],'Months':[Months]})
        Seg_Channel_Hist_Dist=pd.concat([Seg_Channel_Hist_Dist,Hist_Dist],axis=0)

Seg_Channel_Hist_Dist.reset_index(inplace=True,drop=True)

Overall_Channel_Hist_Dist=pd.DataFrame(Final_Model_Data[var_CHANNEL_1].sum())
Overall_Channel_Hist_Dist.reset_index(inplace=True)
Overall_Channel_Hist_Dist.columns=['Channel','Total_Promotion']
Seg_Channel_Hist_Dist=pd.merge(Seg_Channel_Hist_Dist,Overall_Channel_Hist_Dist,
                               on='Channel',how='inner')
Seg_Channel_Hist_Dist['Promotion_Dist_Across_Segment']=Seg_Channel_Hist_Dist['Channel_Promotion']/Seg_Channel_Hist_Dist['Total_Promotion']
Data_Summary_Final=Seg_Channel_Hist_Dist
Data_Summary_Final=Data_Summary_Final[['Segment','Channel','Channel_Promotion','Total_Promotion','Promotion_Dist_Across_Segment','HCP Count','Months']]


sales_lag_names=[]
for i in range(1,var_sales_lags+1) :
    k='total_sales'+"_"+str(i)
    sales_lag_names.append(k)

Sales_lag_Coeff=Parameter_Estimates.loc[Parameter_Estimates['Variable'].isin(sales_lag_names),
                                        ['Segment','Variable','Coeff']]

Sales_lag_Coeff.reset_index(inplace=True,drop=True)

Short_Term_Factor=pd.DataFrame()

for i in range(0,len(segments)) :
    sales_lag=[]
    Monthly_Sales_STF=[]
    for k in range(1,var_sales_lags+1) :
            s='total_sales'+"_"+str(k)
            p=list(Sales_lag_Coeff.loc[(Sales_lag_Coeff['Segment']==segments[i]) & (Sales_lag_Coeff['Variable']==s),'Coeff'])
            sales_lag.extend(p)

    for month in range(0,len(months)) :
            if month==0:
                impact=1
                Monthly_Sales_STF.append(impact)

            else :
                iters=min(month,var_sales_lags)
                impact=1
                for z in range(0,iters) :
                    impact+=Monthly_Sales_STF[month-1-z]*sales_lag[z]
                Monthly_Sales_STF.append(impact)
    Total_STF=sum(Monthly_Sales_STF)/len(months)
    STF=pd.DataFrame({'Segment':[segments[i]],'STF':[Total_STF]})
    Short_Term_Factor=pd.concat([Short_Term_Factor,STF],axis=0)
    Short_Term_Factor.reset_index(drop=True,inplace=True)

Data_Summary_Final=pd.merge(Data_Summary_Final,Short_Term_Factor,on=['Segment'],how='inner')

Long_Term_Factor=pd.DataFrame()

for i in range(0,len(segments)) :
    Monthly_Sales_LTF=[]
    sales_lag = []
    for k in range(1,var_sales_lags+1) :
            s='total_sales'+"_"+str(k)
            p=list(Sales_lag_Coeff.loc[(Sales_lag_Coeff['Segment']==segments[i]) & (Sales_lag_Coeff['Variable']==s),'Coeff'])
            sales_lag.extend(p)

    for month in range(0,36) :
        if month == 0:
            impact=1
            Monthly_Sales_LTF.append(impact)

        elif (~(month==0) & (month<len(months))) :

            iters=min(month,var_sales_lags)
            impact=1
            for z in range(0,iters) :

                impact+=Monthly_Sales_LTF[month-1-z]*sales_lag[z]
            Monthly_Sales_LTF.append(impact)

        else :

            iters=min(month,var_sales_lags)
            impact=0
            for z in range(0,iters) :

                impact+=Monthly_Sales_LTF[month-1-z]*sales_lag[z]
            Monthly_Sales_LTF.append(impact)

    Total_LTF=sum(Monthly_Sales_LTF)/len(months)
    LTF=pd.DataFrame({'Segment' : [segments[i]],'LTF' : [Total_LTF]})
    Long_Term_Factor=pd.concat([Long_Term_Factor,LTF],axis=0)
    Long_Term_Factor.reset_index(drop=True,inplace=True)


Data_Summary_Final=pd.merge(Data_Summary_Final,Long_Term_Factor,on=['Segment'],how='inner')

In [0]:
print(Data_Summary_Final)

In [0]:
################################# ROI Calculation ######################################
Annual_Impact=pd.DataFrame(Final_Impact.groupby(['Segment','Channel'],as_index=False)['Impact'].sum())
Annual_Impact=Annual_Impact[~Annual_Impact['Channel'].isin(['Brand Equity','CarryOver'])]
Annual_Impact.reset_index(inplace=True,drop=True)


Annual_Impact=pd.merge(Annual_Impact,Data_Summary_Final[['Segment','Channel','Channel_Promotion']],how='inner',on=['Segment','Channel'])
Channel_Cost=pd.DataFrame()
for i in range(0,len(var_CHANNEL)) :
    Channel_Name=[var_CHANNEL[i]]
    Channel_Unit_Promotion_Cost=[var_prom_cost_unit_final[i]]
    Cost=pd.DataFrame(zip(Channel_Name,Channel_Unit_Promotion_Cost),
                      columns=['Channel','Cost'])
    Channel_Cost=pd.concat([Channel_Cost,Cost],axis=0)
Channel_Cost.reset_index(inplace=True,drop=True)

Annual_Impact=pd.merge(Annual_Impact,Channel_Cost,how='inner',on=['Channel'])
Annual_Impact['sale_per_unit']=var_sup


LTF_df = Parameter_Estimates.loc[Parameter_Estimates['Variable'].str.contains('total_sales'),
                                ['Segment','Variable','Coeff']]

LTF_df=LTF_df.groupby(['Segment'],as_index=False)['Coeff'].sum()
LTF_df.loc[LTF_df['Coeff']>=1,'Coeff']=0.99
LTF_df['LTF']=1/(1-LTF_df['Coeff'])

ROI_df=pd.merge(Annual_Impact,LTF_df[['Segment','LTF']],how='inner',on=['Segment'])
ROI_df['Long_Term_Impact']=ROI_df['Impact']*ROI_df['LTF']
ROI_df['Short_Term_Sales']=ROI_df['Impact']*ROI_df['sale_per_unit']
ROI_df['Long_Term_Sales']=ROI_df['Long_Term_Impact']*ROI_df['sale_per_unit']
ROI_df['Total_Spend']=ROI_df['Channel_Promotion']*ROI_df['Cost']
ROI_df['Short_Term_ROI(%)']=(ROI_df['Short_Term_Sales']/ROI_df['Total_Spend'])*100
ROI_df['Long_Term_ROI(%)']=(ROI_df['Long_Term_Sales']/ROI_df['Total_Spend'])*100

ROI_df=ROI_df[['Segment','Channel','Short_Term_Sales','Long_Term_Sales','Total_Spend','Short_Term_ROI(%)','Long_Term_ROI(%)']]

National_ROI_df=ROI_df.groupby(['Channel'],as_index=False)[['Short_Term_Sales','Long_Term_Sales','Total_Spend']].sum()
National_ROI_df['Short_Term_ROI(%)']=(National_ROI_df['Short_Term_Sales']/National_ROI_df['Total_Spend'])*100
National_ROI_df['Long_Term_ROI(%)']=(National_ROI_df['Long_Term_Sales']/National_ROI_df['Total_Spend'])*100
National_ROI_df['Segment']='National'
National_ROI_df=National_ROI_df[['Segment','Channel','Short_Term_Sales','Long_Term_Sales','Total_Spend','Short_Term_ROI(%)','Long_Term_ROI(%)']]

Final_ROI_df=pd.concat([ROI_df,National_ROI_df],axis=0)
Final_ROI_df.reset_index(inplace=True,drop=True)


################################ End of ROI Calculations###############################

In [0]:
print(Final_ROI_df)

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

modelling_df = Final_Impact # For this sample code, simply copy input to output


# Write recipe outputs
final_impact = dataiku.Dataset("final_impact")
final_impact.write_with_schema(Final_Impact)

curvature = dataiku.Dataset("curvature")
curvature.write_with_schema(Coeff_Final)

if Loop_Type == "Automated" :
    best_curvature = dataiku.Dataset("best_curvature")
    best_curvature.write_with_schema(Coeff_Best_Model)
else :
    best_curvature = dataiku.Dataset("best_curvature")
    best_curvature.write_with_schema(Coeff_Best_Model_1)

model_parameters = dataiku.Dataset("model_parameters")
model_parameters.write_with_schema(Parameter_Estimates)


national_impact = dataiku.Dataset("national_impact")
national_impact.write_with_schema(Overall_Impact)

Short_Long_Term_Factor = dataiku.Dataset("Short_Long_Term_Factor")
Short_Long_Term_Factor.write_with_schema(Data_Summary_Final)

ROI = dataiku.Dataset("ROI")
ROI.write_with_schema(Final_ROI_df)